# Import statements

Pip installations

In [ ]:
!pip install ultralytics

Import statements

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Directory paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_dir = '/content/drive/MyDrive/Research Files/Other files/Development/Dataset'
segmentation_model = YOLO('/content/drive/MyDrive/Research Files/Other files/Development/YOLOv8 Trained Model/best.pt')

# Global variables

In [1]:
# tf.data.Dataset
MAX_SEQUENCE_LENGTH = 45
BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 100
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

NEW_LENGTH = 291
NEW_WIDTH = 218
IMAGE_CHANNELS = 3

LOSS_FUNCTION = "mean_squared_error"

OPTIMIZER = Adadelta(learning_rate=LEARNING_RATE)
LEARNING_RATE = 1.0

EPOCHS = 50

AUGMENT_PROBABILITY = 0.5

# Functions

In [ ]:
def get_sequences(sequence_dir):
    sequence_paths = sorted(
        [os.path.join(sequence_dir, d) for d in os.listdir(sequence_dir) if os.path.isdir(os.path.join(sequence_dir, d))]
    )
    sequences = []
    for seq_path in sequence_paths:
        frame_paths = sorted(glob.glob(os.path.join(seq_path, '*.*')))  # Adjust the pattern if needed
        sequences.append(frame_paths)
    return sequences

In [ ]:
def create_tf_dataset(sequences):
    dataset = tf.data.Dataset.from_tensor_slices(sequences)
    return dataset

In [ ]:
def preprocess_image(image_path):
    # Load image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # ------------------------------

    # Apply segmentation model
    results = segmentation_model.predict(source=img, save=False, verbose=False)

    # Get the segmentation mask
    masks = results[0].masks
    if masks is not None and len(masks.data) > 0:
        # Assuming the first mask is the plant
        mask = masks.data[0].cpu().numpy()  # Shape: (H, W)
        mask = (mask * 255).astype(np.uint8)
    else:
        # If no mask is found, use an empty mask
        mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)

    # Apply the mask to the image
    segmented_img = cv2.bitwise_and(img, img, mask=mask)

    # ------------------------------

    # Resize the image
    img_resized = cv2.resize(segmented_img, (NEW_LENGTH, NEW_WIDTH))

    # ------------------------------

    # Normalize the image to [0,1]
    img_normalized = img_resized / 255.0

    # ------------------------------

    # Expand dimensions to add the channel dimension
    img_normalized = np.expand_dims(img_normalized, axis=-1)

    return img_normalized.astype(np.float32)

In [ ]:
def preprocess_sequence(frame_paths, max_length):
    frames = []
    for frame_path in frame_paths:
        img = preprocess_image(frame_path)
        frames.append(img)
    frames = np.array(frames)

    # Pad or truncate the sequence to 'max_length'
    sequence_length = frames.shape[0]
    if sequence_length < max_length:
        # Pad with zeros at the beginning
        pad_length = max_length - sequence_length
        padding = np.zeros((pad_length, NEW_WIDTH, NEW_LENGTH, IMAGE_CHANNELS), dtype=np.float32)
        frames = np.concatenate([padding, frames], axis=0)
    else:
        # Truncate the sequence
        frames = frames[-max_length:]
    return frames

In [ ]:
def process_sequence_tf(frame_paths):
    frames = tf.py_function(
        func=preprocess_sequence,
        inp=[frame_paths, MAX_SEQUENCE_LENGTH],
        Tout=tf.float32
    )
    frames.set_shape([MAX_SEQUENCE_LENGTH, NEW_WIDTH, NEW_LENGTH, IMAGE_CHANNELS])
    return frames

In [ ]:
def create_input_target(frames):
    inputs = frames[:-1]   # Exclude the last frame
    targets = frames[1:]   # Exclude the first frame
    return inputs, targets

In [ ]:
def augment_sequence(frames):
    """
    Applies random augmentations to a sequence of frames.

    Args:
        frames: Tensor of shape (sequence_length, height, width, channels)

    Returns:
        Augmented frames of the same shape.
    """
    def _augment(frames_np):
        import numpy as np
        import cv2

        # Decide whether to apply augmentation
        if np.random.rand() < AUGMENT_PROBABILITY:
            # Define augmentation parameters
            # Example: Random flip and rotation
            flip_code = np.random.choice([None, 0, 1, -1])  # None: no flip, 0: vertical, 1: horizontal, -1: both
            angle = np.random.uniform(-15, 15)  # Rotate between -15 to 15 degrees

            # Define the rotation matrix
            if angle != 0:
                center = (IMAGE_WIDTH / 2, IMAGE_HEIGHT / 2)
                rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

            augmented_frames = []
            for frame in frames_np:
                frame = frame.squeeze()  # Shape: (height, width)

                # Apply flip if selected
                if flip_code is not None:
                    frame = cv2.flip(frame, flip_code)

                # Apply rotation if angle is not zero
                if angle != 0:
                    frame = cv2.warpAffine(frame, rotation_matrix, (IMAGE_WIDTH, IMAGE_HEIGHT), borderMode=cv2.BORDER_REFLECT)

                # Expand dims to add channel back
                frame = np.expand_dims(frame, axis=-1)
                augmented_frames.append(frame)

            augmented_frames = np.array(augmented_frames)
            return augmented_frames.astype(np.float32)
        else:
            # No augmentation applied
            return frames_np

    # Use tf.py_function to apply the augmentation
    augmented_frames = tf.py_function(func=_augment, inp=[frames], Tout=tf.float32)

    # Set the shape
    augmented_frames.set_shape([MAX_SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    return augmented_frames

# Import dataset

## Load dataset

Verify if dataset has been loaded

## Split training and validation sets

In [ ]:
train_sequences_dir = os.path.join(dataset_dir, 'train_set')
val_sequences_dir = os.path.join(dataset_dir, 'val_set')
test_sequences_dir = os.path.join(dataset_dir, 'test_set')

In [ ]:
train_sequences = get_sequences(train_sequences_dir)
val_sequences = get_sequences(val_sequences_dir)
test_sequences = get_sequences(test_sequences_dir)

In [ ]:
train_dataset = create_tf_dataset(train_sequences)
val_dataset = create_tf_dataset(val_sequences)
test_dataset = create_tf_dataset(test_sequences)

In [ ]:
train_dataset = train_dataset.map(process_sequence_tf, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(process_sequence_tf, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(process_sequence_tf, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Apply data augmentation only to the training set
train_dataset = train_dataset.map(lambda x: augment_sequence(x), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_dataset = train_dataset.map(create_input_target, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(create_input_target, num_parallel_calls=tf.data.AUTOTUNE)

Verify if training and validation sets are splitted

# Segment plant object

# Preprocess dataset

In [ ]:
# For the training dataset
train_dataset = (
    train_dataset
    .cache()
    .shuffle(SHUFFLE_BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(PREFETCH_BUFFER_SIZE)
)

val_dataset = (
    val_dataset
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(PREFETCH_BUFFER_SIZE)
)

test_dataset = (
    test_dataset
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(PREFETCH_BUFFER_SIZE)
)

Verify shapes

In [ ]:
# Example of verifying shapes
for inputs, targets in train_dataset.take(1):
    print('Inputs shape:', inputs.shape)   # Expected: (BATCH_SIZE, sequence_length, 218, 291, 1)
    print('Targets shape:', targets.shape) # Expected: (BATCH_SIZE, sequence_length, 218, 291, 1)

# Develop model

## *Iteration X*

### Build model

### Visualize model

### Compile model

### Train model

### Plot results

## *Iteration 1*

### Build model

In [ ]:
model = models.Sequential([
    layers.Input(shape=(None, NEW_WIDTH, NEW_LENGTH, IMAGE_CHANNELS)),
    layers.ConvLSTM2D(
        filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
    ),
    layers.BatchNormalization(),
    layers.ConvLSTM2D(
        filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
    ),
    layers.BatchNormalization(),
    layers.ConvLSTM2D(
        filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
    ),
    layers.BatchNormalization(),
    layers.ConvLSTM2D(
        filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
    ),
    layers.BatchNormalization(),
    layers.Conv3D(
        filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
    ),
])

### Visualize model

In [ ]:
model.summary()

### Compile model

In [ ]:
model.compile(loss=LOSS_FUNCTION, optimizer=OPTIMIZER)

### Train model

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

### Plot results

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

Visualize results

In [ ]:
# Select a sequence from the training set
example_sequence_paths = train_sequences[0]  # Adjust index as needed
example_sequence = preprocess_sequence(example_sequence_paths, MAX_SEQUENCE_LENGTH)

# Prepare inputs
example_input = example_sequence[:-1]  # Inputs
example_target = example_sequence[1:]  # Ground truth

# Expand dimensions to simulate batch size of 1
example_input = np.expand_dims(example_input, axis=0)  # Shape: (1, sequence_length, 40, 40, 1)

# Predict the sequence
predicted_sequence = model.predict(example_input)
predicted_sequence = predicted_sequence[0]  # Remove batch dimension

# Visualize the results
num_frames = 5  # Number of frames to display

plt.figure(figsize=(15, 6))

# Original frames
for i in range(num_frames):
    plt.subplot(2, num_frames, i + 1)
    plt.imshow(example_sequence[i])
    plt.title(f'Frame {i + 1} (Original)')
    plt.axis('off')

# Predicted frames
for i in range(num_frames):
    plt.subplot(2, num_frames, num_frames + i + 1)
    if i < 3:
        # Show original frames for the first 3 frames
        plt.imshow(example_sequence[i])
        plt.title(f'Frame {i + 1} (Original)')
    else:
        # Show predicted frames for subsequent frames
        plt.imshow(example_sequence[i])
        plt.title(f'Frame {i + 1} (Predicted)')
    plt.axis('off')

plt.tight_layout()
plt.show()

# Test model